### Import Libs and load creds

In [1]:
import os
import json
from typing import List
from langchain_groq import ChatGroq
import pandas as pd
# from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
from pydantic import BaseModel,Field


llm = ChatGroq(api_key=os.getenv("GROQ_API_KEY"),
                 model="llama3-8b-8192"
                 
                )

neo4j_url = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)

C:\Users\COMP\AppData\Local\Temp\ipykernel_15036\3675335738.py:20: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relat

In [26]:
llm.invoke("Hi, explain me about cypher query language").content

'Cypher is a query language specifically designed for graph databases, such as Neo4j. It\'s used to retrieve, manipulate, and analyze data stored in a graph structure. Here\'s an overview of Cypher:\n\n**What is Cypher?**\n\nCypher is a declarative language, which means you specify what you want to retrieve or manipulate, rather than how to do it. It\'s similar to SQL (Structured Query Language) for relational databases, but designed for graph databases.\n\n**Cypher syntax**\n\nCypher queries consist of a series of clauses, which are used to specify the query\'s intent. The basic syntax is as follows:\n```cypher\nMATCH (pattern) [WITH clause] [RETURN clause]\n```\nHere:\n\n* `MATCH` clause: Specifies the pattern to match in the graph.\n* `WITH` clause (optional): Allows you to alias or manipulate the matched nodes or relationships.\n* `RETURN` clause: Specifies the data to be returned.\n\n**Cypher patterns**\n\nCypher patterns are used to specify the relationships between nodes in the 

### Pydantic class model

In [27]:
class Question(BaseModel):
    questions: List[str] = Field(
        description="List of relevant questions for the particular graph schema. Make sure that questions can be answered with information from the schema and that the questions are diverse as possible. Make sure that the schema and the example values contains the information that can answer the questions! Do not ask questions that cannot be answered based on the provided schema. For example, if no information about subtitles can be found in the graph, don't ask any information about subtitles. Make sure to always limit the results to less than 10 results by saying 3 users, or top 5 movies, or similar."
    )

structured_llm = llm.with_structured_output(Question)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


system_prompt = """Your task is to generate 1000 questions that are directly related to a specific graph schema in Neo4j. Each question should target distinct aspects of the schema, such as relationships between nodes, properties of nodes, or characteristics of node types. Ensure that the questions vary in complexity, covering basic, intermediate, and advanced queries.

Avoid ambiguous questions. For clarity, an ambiguous question is one that can be interpreted in multiple ways or does not have a straightforward answer based on the schema. For example, avoid asking, "What is related to this?" without specifying the node type or relationship.
Please design each question to yield a limited number of results, specifically between 10 to 50 results. This will ensure that the queries are precise and suitable for detailed analysis and training.
The goal of these questions is to create a dataset for training AI models to convert natural language queries into Cypher queries effectively.
It is vital that the database contains information that can answer the question!
Make sure to generate 1000 questions!"""

default_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            f"{system_prompt} Follow these instructions create minimum 100 possible questions: {{instructions}}"
        ),
        HumanMessagePromptTemplate.from_template(
            "Make sure to create questions for the following graph schema:{input}\n Here are some example nodes and relationship values: {values}. Don't use any values that aren't found in the schema or in provided values. Also, do not ask questions that there is no way to answer based on the schema or provided example values. Don't include question index or the sequence of the question in the list. Make sure your question is complete and clear"
        ),
    ]
)

chain = default_prompt | structured_llm

### Specifying query types using dictionary

In [28]:
query_types = {
    "Simple Retrieval Queries": "These queries focus on basic data extraction, retrieving nodes or relationships based on straightforward criteria such as labels, properties, or direct relationships. Examples include fetching all nodes labeled as 'Person' or retrieving relationships of a specific type like 'EMPLOYED_BY'. Simple retrieval is essential for initial data inspections and basic reporting tasks. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Retrieval Queries": "These advanced queries use the rich pattern-matching capabilities of Cypher to handle multiple node types and relationship patterns. They involve sophisticated filtering conditions and logical operations to extract nuanced insights from interconnected data points. An example could be finding all 'Person' nodes who work in a 'Department' with over 50 employees and have at least one 'REPORTS_TO' relationship. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Simple Aggregation Queries": "Simple aggregation involves calculating basic statistical metrics over properties of nodes or relationships, such as counting the number of nodes, averaging property values, or determining maximum and minimum values. These queries summarize data characteristics and support quick analytical conclusions. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Pathfinding Queries": "Specialized in exploring connections between nodes, these queries are used to find the shortest path, identify all paths up to a certain length, or explore possible routes within a network. They are essential for applications in network analysis, routing, logistics, and social network exploration. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Aggregation Queries": "The most sophisticated category, these queries involve multiple aggregation functions and often group results over complex subgraphs. They calculate metrics like average number of reports per manager or total sales volume through a network, supporting strategic decision making and advanced reporting. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Verbose query": "These queries are characterized by their explicit and detailed specifications about the data retrieval process and the exact information needed. They involve elaborate instructions for navigating through complex data structures, specifying precise criteria for inclusion, exclusion, and sorting of data points. Verbose queries typically require the breakdown of each step in the querying process, from the initial identification of relevant data nodes and relationships to the intricate filtering and sorting mechanisms that must be applied. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
}

### Connect to Neo4j

In [29]:
graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)
schema = graph.schema
all_questions = []
for type in query_types:
  print(type)
  instructions = f"{type}: {query_types[type]}"
  values = graph.query(
            """
            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2
            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}
            RETURN p
            """
            )
  questions = chain.invoke( {"input": schema, "instructions": instructions, "values": values})
  all_questions.extend([{"question": el, "type": type} for el in questions.questions])

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 3, column: 13, offset: 69} for query: '\n            MATCH (n) WHERE ran

Simple Retrieval Queries


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 3, column: 13, offset: 69} for query: '\n            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2\n            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}\n            RETURN p\n            '


Complex Retrieval Queries


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 3, column: 13, offset: 69} for query: '\n            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2\n            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}\n            RETURN p\n            '


...

Pathfinding Queries


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 3, column: 13, offset: 69} for query: '\n            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2\n            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}\n            RETURN p\n            '


Complex Aggregation Queries


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 3, column: 13, offset: 69} for query: '\n            MATCH (n) WHERE rand() > 0.6 WITH n LIMIT 2\n            CALL { WITH n MATCH p=(n)-[*3..3]-() RETURN p LIMIT 1}\n            RETURN p\n            '


Verbose query


In [30]:

import random

random_elements = random.sample(all_questions, 10)

print(random_elements)

[{'question': 'What is the average overall rating of players who play as defenders?', 'type': 'Verbose query'}, {'question': 'What is the name of the club a player named Ronaldinho plays for?', 'type': 'Pathfinding Queries'}, {'question': 'Which players have a release clause above 10,000,000?', 'type': 'Complex Retrieval Queries'}, {'question': 'What is the name of the club with the highest number of players with a wage above 20,000?', 'type': 'Simple Retrieval Queries'}, {'question': 'What is the average overall rating of players who play as goalkeepers?', 'type': 'Verbose query'}, {'question': 'Which players have a performance best overall above 80?', 'type': 'Complex Retrieval Queries'}, {'question': 'Which players have a performance best overall below 70?', 'type': 'Complex Retrieval Queries'}, {'question': 'What is the wage of a player who has a potential of 83.0 and plays for a club named Querétaro and has an overall rating above 80 and plays as a CAM?', 'type': 'Pathfinding Quer

In [33]:
all_questions_df = pd.DataFrame.from_records(all_questions)
all_questions_df.to_csv('../processed_data/text2cypher_questions.csv', index = False)

In [34]:
class CypherQuery(BaseModel):
    cypherquery: str = Field(
        description="A correct Neo4J Cypher Query Language without any preambles. Make sure that it follows the schema"
    )


structured_llm_cypher_query = llm.with_structured_output(CypherQuery)

system_prompt_cypher_query = """Given an input question, convert it to a Cypher query. No pre-amble.
Additional instructions:
- Ensure that queries checking for non-null properties use `IS NOT NULL` in a straightforward manner.
- Don't use `size((n)--(m))` for counting relationships. Instead use the new `count{{(n)--(m))}}` syntax.
- Incorporate the new existential subqueries in examples where the query needs to check for the existence of a pattern.
  Example: MATCH (p:Person)-[r:IS_FRIENDS_WITH]->(friend:Person)
            WHERE exists{{ (p)-[:WORKS_FOR]->(:Company {{name: 'Neo4j'}})}}
            RETURN p, r, friend"""

default_prompt_cypher_query = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            f"{system_prompt_cypher_query}"
        ),
        HumanMessagePromptTemplate.from_template(
            """Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question: {schema}"
              Question: {question}
              Cypher query:"""),
    ]
)

chain_cypher_query = default_prompt_cypher_query | structured_llm_cypher_query
     

In [35]:
chain_cypher_query.invoke({"schema": schema, "question":"Whats the team that Vini Jr plays for?"}).cypherquery

"MATCH (p:Player)-[r:PLAYS_FOR]->(c:Club) WHERE p.name = 'Vini Jr' RETURN c.name"

In [36]:
import time

graph = Neo4jGraph(url=neo4j_url,username=neo4j_user,password=neo4j_password,sanitize=True,enhanced_schema=True)
schema = graph.schema

def get_cypher_query(question):
    cypher_query = chain_cypher_query.invoke({"schema": schema, "question": question}).cypherquery
    time.sleep(1)
    return cypher_query

def safe_get_cypher_query(question):
    retry_count = 3
    for attempt in range(retry_count):
        try:
            result = get_cypher_query(question)
            print(question, ':', result)
            return result
        except Exception as e:
            print(f"Error: {e}. Retrying ({attempt + 1}/{retry_count})...")
            time.sleep(random.uniform(1, 3))
    return None

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [14]:
df['cypher'] = df['question'].apply(lambda q: safe_get_cypher_query(q))

What is the name of the player with the highest overall rating and the highest potential rating and the highest wage and the highest release clause value and the highest overall rating and the highest potential rating and the highest release clause value and the highest overall rating and the highest potential rating? : MATCH (p:Player)
WITH p, max(p.overall_rating) as max_overall, max(p.potential) as max_potential, max(p.wage) as max_wage, max(p.release_clause) as max_release_clause
WHERE p.overall_rating = max_overall AND p.potential = max_potential AND p.wage = max_wage AND p.release_clause = max_release_clause
RETURN p.name
What is the name of the player with the highest overall rating and the highest potential rating and the highest wage and the highest release clause value and the highest overall rating and the highest potential rating and the highest release clause value and the highest overall rating and the highest potential rating and the highest release clause value? : MATCH

NameError: name 'random' is not defined

In [ ]:
# df.to_excel('../processed_data/text2cypher_questions_with_cypher.xlsx',index=False)

In [2]:
df=pd.read_excel(r'../processed_data/text2cypher_questions_with_cypher.xlsx')
df

,question,type,cypher
0,Which clubs have the most players with an over...,Simple Aggregation Queries,MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE ...
1,Which clubs have the most players with a value...,Simple Aggregation Queries,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[..."
2,Which clubs have the most players with a best ...,Simple Aggregation Queries,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[..."
3,What is the average wage of all players with a...,Simple Aggregation Queries,MATCH (p:Player) WHERE p.best_overall > 85 MAT...
4,Which players have a best overall rating above...,Simple Aggregation Queries,MATCH (p:Player) WHERE p.best_overall > 85 RET...
...,...,...,...
213,What is the average wage of players who are pl...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position {...
214,What is the average overall rating of players ...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position)-...
215,What is the average wage of players who are pl...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position)<...
216,Which players have a wage above 5000 and are n...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_FOR]->(c:Club) WHERE...


### Evaluate the generated Cypher query by running it against the database. Check if it has any syntax errors, times out, or returns a result.

In [3]:
graph = Neo4jGraph(url=neo4j_url, username=neo4j_user, password=neo4j_password, sanitize=True, enhanced_schema=True)

# Define the function to execute the query and handle errors
def execute_query(row):
    try:
        # Execute the Cypher query from the 'cypher' column
        data = graph.query(row["cypher"])
        
        # Check if the query returned data
        if data:
            return pd.Series([True, False, False])  # returns_results, syntax_error, timeout_error
        else:
            return pd.Series([False, False, False])  # No results, no error, no timeout
    
    except ValueError as e:
        # Handle invalid Cypher query errors
        if "Generated Cypher Statement is not valid" in str(e):
            print(f"Syntax error in Cypher query: {e}")
            return pd.Series([False, True, False])  # False for results, True for syntax error, no timeout
        else:
            print(f"Other ValueError: {e}")
            return pd.Series([False, False, False])  # False for results, no error, no timeout
    
    except Exception as e:
        # Handle timeout errors
        if hasattr(e, 'code') and e.code == "Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration":
            return pd.Series([False, False, True])  # False for results, no syntax error, True for timeout
        else:
            # Handle other exceptions
            print(f"Unexpected error: {e}")
            return pd.Series([False, False, False])  # False for results, no error, no timeout

# Apply the function to the 'cypher' column and unpack the returned Series into new columns
df[['query_result', 'syntax_error', 'timeout_error']] = df.apply(execute_query, axis=1)
df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Unexpected error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'nan': expected 'FOREACH', 'ALTER', 'ORDER BY', 'CALL', 'USING PERIODIC COMMIT', 'CREATE', 'LOAD CSV', 'START DATABASE', 'STOP DATABASE', 'DEALLOCATE', 'DELETE', 'DENY', 'DETACH', 'DROP', 'DRYRUN', 'FINISH', 'GRANT', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REALLOCATE', 'REMOVE', 'RENAME', 'RETURN', 'REVOKE', 'ENABLE SERVER', 'SET', 'SHOW', 'SKIP', 'TERMINATE', 'UNWIND', 'USE' or 'WITH' (line 1, column 1 (offset: 0))
"nan"
 ^}
Unexpected error: {code: Neo.ClientError.Statement.SyntaxError} {message: A pattern expression should only be used in order to test the existence of a pattern. It can no longer be used inside the function size(), an alternative is to replace size() with COUNT {}. (line 1, column 176 (offset: 175))
"MATCH (c:Club)-[:PLAYS_FOR]->(p:Player)-[:PLAYS_AS]->(pos:Position)<-[:HAS_PERFORMANCE]-(perf:Performance) WHERE perf.overall_rating > 75 AND pos.name

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: HAS_FOOT)} {position: line: 1, column: 48, offset: 47} for query: "MATCH (p:Player)-[:PLAYS_AS]->(pos:Position)-[:HAS_FOOT]->(foot:Foot {type:'Right'})-[:HAS_FINANCIALS]->(fin:Financials) WHERE p.fin.value > 8500000 RETURN p"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you di

Unexpected error: {code: Neo.ClientError.Statement.SyntaxError} {message: Query cannot conclude with MATCH (must be a RETURN clause, a FINISH clause, an update clause, a unit subquery call, or a procedure call with no YIELD). (line 1, column 1 (offset: 0))
"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club)"
 ^}
Unexpected error: {code: Neo.ClientError.Statement.SyntaxError} {message: Query cannot conclude with MATCH (must be a RETURN clause, a FINISH clause, an update clause, a unit subquery call, or a procedure call with no YIELD). (line 1, column 1 (offset: 0))
"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club)"
 ^}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: AGE_GROUP)} {position: line: 1, column: 89, offset: 88} for query: "MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE p.release_clause > 12300000 AND NOT (p)-[:AGE_GROUP]->(:AgeGroup {name: 'Young Adult'}) RETURN p"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell 

Unexpected error: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '{': expected an expression, 'FOREACH', 'ORDER BY', 'CALL', 'CREATE', 'LOAD CSV', 'DELETE', 'DETACH', 'FINISH', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REMOVE', 'RETURN', 'SET', 'SKIP', 'UNION', 'UNWIND', 'USE', 'WITH' or <EOF> (line 1, column 189 (offset: 188))
"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club)-[:PLAYS_AS]->(pos:Position)-[:HAS_ROLE]->(role:Role)-[:HAS_PERFORMANCE]->(perf:Performance)-[:HAS_FINANCIALS]->(fin:Financials) WHERE p.age:AgeGroup{name:'Prime'} AND fin.value > 1.5E8 RETURN p"
                                                                                                                                                                                             ^}


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PLAYS_IN)} {position: line: 1, column: 20, offset: 19} for query: "MATCH (p:Player)-[:PLAYS_IN {age_group: 'Veteran'}]->() WITH avg(p.overall_rating) as average_rating RETURN average_rating"
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label 

,question,type,cypher,query_result,syntax_error,timeout_error
0,Which clubs have the most players with an over...,Simple Aggregation Queries,MATCH (p:Player)-[:PLAYS_FOR]->(c:Club) WHERE ...,True,False,False
1,Which clubs have the most players with a value...,Simple Aggregation Queries,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[...",True,False,False
2,Which clubs have the most players with a best ...,Simple Aggregation Queries,"MATCH (p:Player)-[:PLAYS_FOR]->(c:Club), (p)-[...",True,False,False
3,What is the average wage of all players with a...,Simple Aggregation Queries,MATCH (p:Player) WHERE p.best_overall > 85 MAT...,True,False,False
4,Which players have a best overall rating above...,Simple Aggregation Queries,MATCH (p:Player) WHERE p.best_overall > 85 RET...,True,False,False
...,...,...,...,...,...,...
213,What is the average wage of players who are pl...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position {...,True,False,False
214,What is the average overall rating of players ...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position)-...,True,False,False
215,What is the average wage of players who are pl...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_AS]->(pos:Position)<...,True,False,False
216,Which players have a wage above 5000 and are n...,Complex Retrieval Queries,MATCH (p:Player)-[r:PLAYS_FOR]->(c:Club) WHERE...,True,False,False


In [4]:
df[df['query_result']==True]['type'].value_counts()

type
Simple Aggregation Queries    106
Complex Retrieval Queries      35
Simple Retrieval Queries       23
Name: count, dtype: int64

In [7]:
df[df['query_result']==False].reset_index(drop=True)

,question,type,cypher,query_result,syntax_error,timeout_error
0,Which clubs have the most players with a finan...,Simple Aggregation Queries,MATCH (c:Club)-[:PLAYS_FOR]->(p:Player)-[:HAS_...,False,False,False
1,Which clubs have more than 3 players with a be...,Simple Retrieval Queries,NaN,False,False,False
2,Which clubs have more than 2 players with a ov...,Simple Retrieval Queries,MATCH (c:Club)<-[:PLAYS_FOR]-(p:Player)-[:HAS_...,False,False,False
3,Which clubs have more than 2 players with a ov...,Simple Retrieval Queries,MATCH (c:Club)-[:PLAYS_FOR]->(p:Player)-[:PLAY...,False,False,False
4,Which clubs have more than 2 players with a ov...,Simple Retrieval Queries,MATCH (c:Club)-[:PLAYS_FOR]->(p:Player)-[:PLAY...,False,False,False
5,What are the names of all players who have a o...,Simple Retrieval Queries,MATCH (p:Player)-[:HAS_ROLE]->(ageGroup:AgeGro...,False,False,False
6,What is the name of the player with the highes...,Complex Retrieval Queries,MATCH (p:Player) WHERE p.potential = (SELECT m...,False,False,False
7,Which positions have the most players with a v...,Simple Aggregation Queries,MATCH (p:Player)-[:HAS_FINANCIALS]->(f:Financi...,False,False,False
8,What is the name of the player with the highes...,Complex Retrieval Queries,MATCH (p:Player)-[:HAS_FINANCIALS]->(f:Financi...,False,False,False
9,Which players have a financial value above 3.5...,Complex Retrieval Queries,MATCH (p:Player)-[:HAS_FINANCIALS]->(f:Financi...,False,False,False


In [8]:
df.to_excel('text2cypher_questions_with_cypher_validated.xlsx',index=False)